<a href="https://colab.research.google.com/github/Rudancing/New_test/blob/Statistic/%D0%98%D1%82%D0%BE%D0%B3%D0%BE%D0%B2%D0%B0%D1%8F_%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D0%B0_Phyton_%D0%B4%D0%BB%D1%8F_%D0%B0%D0%BD%D0%B0%D0%BB%D0%B8%D0%B7%D0%B0_%D0%B4%D0%B0%D0%BD%D0%BD%D1%8B%D1%85.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

In [ ]:
# загружаем исходный файл
df = pd.read_csv('HR.csv')
# проверяем на пропуски
(df.isna().mean()*100).round(2)

In [ ]:
# проверяем типы данных
df.info()

In [ ]:
# рассчитываем основные статистики для переменных
df.describe()

In [ ]:
#к основным показателям, рассчитанным выше добавляем моду
mode_satisfaction_level = df['satisfaction_level'].mode()[0]
mode_last_evaluation = df['last_evaluation'].mode()[0]
mode_number_project = df['number_project'].mode()[0]
mode_average_montly_hours = df['average_montly_hours'].mode()[0]
mode_time_spend_company = df['time_spend_company'].mode()[0]
data_mode = {'satisfaction_level': mode_satisfaction_level, 'last_evaluation': mode_last_evaluation,
             'number_project': mode_number_project, 'average_montly_hours': mode_average_montly_hours,
             'time_spend_company': mode_time_spend_company }
df_mode = pd.DataFrame(data_mode, index = ['mode'])
df_mode

In [ ]:
# строим корреляционную матрицу
round(df.corr(),3)

In [ ]:
# визуализируем при помощи тепловой карты
sns.set(rc={'figure.figsize':(8,6)})
sns.heatmap(round(df.corr(),3) , annot = True, cmap = 'magma')

In [ ]:
# определяем, что
# две наиболее скоррелированные переменные:
# Среднее количество часов на рабочем месте в месяц и Количество проектов, выполненных за время работы (0,42)
# две наименее скоррелированные переменные:
# Происходили ли несчастные случаи на рабочем месте с сотрудником и Стаж работы в компании в годах (0,002)л

In [ ]:
# считаем количество сотрудников по отделам
df_employees_by_dept = df.groupby(['department'])['salary'].count().reset_index()
df_employees_by_dept.rename(columns ={'salary': '№ of employees'}, inplace = True )
df_employees_by_dept

In [ ]:
# визуализируем распределение сотрудников по уровню зп
df_salary_bar = df.groupby(['salary'])['department'].count()
df_salary_bar.plot(kind = 'bar', title = 'Распределение сотрудников по зарплатам', xlabel = 'уровень зарплаты', ylabel = 'кол-во сотрудников')
for i, val in enumerate(df_salary_bar.values):
    plt.text(i, val, int(val), horizontalalignment = 'center', verticalalignment = 'bottom', fontdict = {'size': 10})
    print(i, val)

In [ ]:
# Для визуализации распределения сотрудников по уровню зп по департаментам выбрал столбчатую диаграмму с накоплением
df_salary_by_dept_bar = df.groupby(['department','salary'])['left'].count().reset_index()
df_pivot_salary = pd.pivot_table(df_salary_by_dept_bar.reset_index(), index='department', columns='salary', values='left')
df_pivot_salary.plot(kind = 'bar', title = 'Распределение сотрудников по зарплатам по отделам', xlabel = 'департамент', ylabel = 'кол-во сотрудников', stacked = True)
for i, val in enumerate(df_pivot_salary.values):
    pos = 0
    for el in val:
      pos = pos + el
      plt.text(i, pos, '{:.1%}'.format(el/sum(val)), horizontalalignment = 'center', verticalalignment = 'top', fontdict = {'size': 10})

In [ ]:
# Выдвигаем гипотезу, что сотрудники с высоким окладом проводят на работе больше времени, чем сотрудники с низким окладом.
# В этом случае нашей нулевой гипотезой будет
# H0 - нет зависмости размера оклада от времени проведенного на работе
# H1 - зависимость размера оклада от времени проведенного на работе существует

In [ ]:
# Для начала формируем два датасэта для сотрудников и низким окладом и для сотрудников и высоким окладом.
df_low_salary = df[['average_montly_hours', 'salary']][df['salary'] == 'low']
df_high_salary =df[['average_montly_hours', 'salary']][df['salary'] == 'high']
# принимаем уровень значимости а = 0.05
alpha = 0.05

In [ ]:
# Смотрим на распределенеи обеих выборок и выбросы
df_low_salary.plot(kind = 'hist', title = 'Распределение ср. кол-ва часов на работе в мес (low salary)', bins = 60, density = True)
df_high_salary.plot(kind = 'hist', title = 'Распределение ср. кол-ва часов на работе в мес (high salary)', bins = 60, density = True)
df_low_salary.plot(kind = 'box', grid = True)
df_high_salary.plot(kind = 'box', grid = True)

In [ ]:
# распределение в обеих выборках нормальное, выбросы отсутствуют, применяем тест Стьюдента
stat, p = stats.ttest_ind(df_low_salary['average_montly_hours'], df_high_salary['average_montly_hours'])
print('pvalue = ', p)

if (p < alpha):
  print ('p-value меньше нашего уровня значимости - принимаем альтернативную гипотезу H1/')
else:
  print ('p-value больше нашего уровня значимости - нулевую гипотезу H0 не отклоняем.')

In [ ]:
# В результате проверки гипотезы, мы не можем утверждать, что сотрудники с высоким окладом
# проводят на работе больше времени, чем сотрудники с низким окладом. Гипотеза не подтверждена.

In [ ]:
# Делимм датасет на два: уволившиеся и неуволившиеся сотрудники
df_left = df[df['left'] == 1]
df_stayed = df[df['left'] == 0]
# считаем долю сотрудников с повышением за последние 5 лет
promotion_share_left = df_left['promotion_last_5years'].sum() / df_left['promotion_last_5years'].count()
promotion_share_stayed = df_stayed['promotion_last_5years'].sum() / df_stayed['promotion_last_5years'].count()
# считаем среднюю удовлетворенность и кол-во проектов для обеих выборок
satisfaction_left = df_left['satisfaction_level'].mean()
satisfaction_stayed = df_stayed['satisfaction_level'].mean()
projects_left = df_left['number_project'].mean()
projects_stayed = df_stayed['number_project'].mean()
print('\033[1m''Доля сотрудников с повышением за последние 5 лет составляет:''\033[0m''\nдля уволившихся сотрудников: {:.1%}'.format(promotion_share_stayed), '\nдля оставшихся сотрудников: {:.1%}'.format(promotion_share_left))
print('\033[1m''Средняя удовлетворенность:''\033[0m''\nдля уволившихся сотрудников: {:.1%}'.format(satisfaction_left), '\nдля оставшихся сотрудников: {:.1%}'.format(satisfaction_stayed))
print('\033[1m''Среднее кол-во проектов:''\033[0m''\nдля уволившихся сотрудников:', round(projects_left,0), '\nдля оставшихся сотрудников:', round(projects_stayed,0))

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [ ]:
# разделяем выборку на тестовую и обучающую и строим модель регрессии
x = df[['satisfaction_level', 'last_evaluation', 'number_project',
        'average_montly_hours', 'time_spend_company', 'Work_accident', 'promotion_last_5years']]
y = df['left']

x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=42, test_size = 0.3)

In [ ]:
lr = LinearRegression()

In [ ]:
lr.fit(x_train, y_train)

LinearRegression()

In [ ]:
lr.coef_

In [ ]:
lr.intercept_

In [ ]:
y_pred = lr.predict(x_test)

In [ ]:
# посмотрим основные метрики
print(f'R2: {metrics.r2_score(y_test, y_pred)}')
print(f'MAE: {metrics.mean_absolute_error(y_test, y_pred)}')
print(f'MSE: {metrics.mean_squared_error(y_test, y_pred)}')
print(f'RMSE: {np.sqrt(metrics.mean_squared_error(y_test, y_pred))}')

In [ ]:
sns.histplot((y_test - y_pred), kde = True, bins = 20)

In [ ]:
# так как значения y_test содержат только 0 и 1, то попробуем модифицировать полученные значения y_pred, таким образом, чтобы в них также были только 0 и 1.
# а после еще раз рассчитаем метрики
y_pred_mod = [int(round(el)) for el in y_pred]
print(f'R2: {metrics.r2_score(y_test, y_pred_mod)}')
print(f'MAE: {metrics.mean_absolute_error(y_test, y_pred_mod)}')
print(f'MSE: {metrics.mean_squared_error(y_test, y_pred_mod)}')
print(f'RMSE: {np.sqrt(metrics.mean_squared_error(y_test, y_pred_mod))}')

In [ ]:
sns.histplot((y_test - y_pred_mod), kde = True, bins = 20)

In [ ]:
# после модификации качество модели повысилось, но тем не менее
# на основании полученных метрик, в частности коэффициента детерминации можно судить о неудовлетворительном качестве построенной модели.